# Spark Advanced - Joins

## Мотивация

При работе с Apache Spark соединения (Joins) являются одной из самых популярных операций, поэтому очень важно понимать как Apache Spark выполняет такие запросы, а также уметь анализировать результаты исполнения запросов.

## Запуск приложения

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark import StorageLevel

### Создание сессии

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Joins")
        .master("local[4]")
        .config("spark.sql.autoBroadcastJoinThreshold", "-1")
        .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
        .getOrCreate()
)
sc = spark.sparkContext

Spark UI доступен на порту [4040](http://localhost:4040)

### Подготовка данных

In [ ]:
! cd /tmp && rm -rf steam && rm -rf /tmp/spark-warehouse && unzip ~/work/data/steam.zip

In [ ]:
sc.setJobDescription("Разбить датафреймы на 8 партиции")
spark.read.parquet("/tmp/steam/games.parquet").repartition(4).write.mode("overwrite").parquet("/tmp/steam_partitions/games")
spark.read.parquet("/tmp/steam/details.parquet").repartition(4).write.mode("overwrite").parquet("/tmp/steam_partitions/details")
spark.read.parquet("/tmp/steam/tags.parquet").repartition(4).write.mode("overwrite").parquet("/tmp/steam_partitions/tags")

In [ ]:
sc.setJobDescription("Загрузка датафрейма")
games = spark.read.parquet("/tmp/steam_partitions/games")
details = spark.read.parquet("/tmp/steam_partitions/details")
tags = spark.read.parquet("/tmp/steam_partitions/tags")

In [ ]:
games.printSchema()

In [ ]:
details.printSchema()

In [ ]:
tags.printSchema()

## Оптимизация через анализ потребления ресурсов

Два одинаковых по смыслу запроса могут показывать разницу в потреблении ресурсов. В качестве примера рассмотрим два запрос, которые отличаются лишь порядком соединения таблиц:

In [ ]:
sc.setJobDescription("Соединение `details`, `tags`, `games`")
result = details \
  .join(tags, "tag") \
  .join(games, "app_id")
result.count()

In [ ]:
sc.setJobDescription("Соединение `details`, `games`, `tags`")
result = details \
  .join(games, "app_id") \
  .join(tags, "tag")
result.count()

Планы запросов доступны в Spark UI на вкладке [SQL / DataFrame](http://localhost:4040/SQL/).

Анализируя планы запросов, можно заключить следующее:

- с диска читаются одни и те же файлы, но уже на первом перемешивании объем данных отличается по причине того, что соединяются разные таблицы:
    - в первом запросе соединяются таблицы `details` и `tags` по колонке `tag` (ключ партиционирования),
    - во втором запросе соединяются таблицы `details` и `games` по колонке `app_id` (ключ партиционирования).

![](../imgs/spark-join-details-games-tags-exchange1.drawio.svg)

- сортировка перед первым слиянием выполняется на разном объеме данных, что ведет к разному потреблению памяти;
- на втором перемешивании партиционирование выполняется:
    - по `app_id` в пером запросе,
    - по `tag` во втором запросе.

![](../imgs/spark-join-details-games-tags-exchange2.drawio.svg)

- партиционирование по `tag` генерирует больше данных для перемешивания.

Сравнение планов запросов с отображением основных отличий по таблице `details`:

![](../imgs/spark-compare-plans.drawio.svg)

## Порядок соединений (JOIN)

На примере выше было продемонстрировано, что Apache Spark не меняет порядок соединений таблиц: в каком порядке таблицы соеденены в тексте запроса, в том порядке они и будут соединены во время исполнения. Но одна из основных стратегий оптимизации заключается в определении наиболее оптимального порядка соединения таблиц.

### Почему порядок соединений (JOIN) важен?

Рассмотрим следующий пример.

Дано:

1. Таблица `details` содержит 540 тысяч строк,
1. Таблица `games` содержит 50 тысяч строк,
1. Таблица `tags` содержит 441 строку,
1. в таблице `games` только 12 тысяч игр поддерживают Mac OS.

**Найти** оптимальный порядок join.

#### Порядок `details`, `tags`, `games`

1. после соединения `details` и `tags` получится 540 тысяч строк,
1. результат предыдущего соединения соединяется с предварительно отфильтрованной таблицей `games` по признаку `mac`.

В результате получится 145 тысяч строк, но на промежуточном этапе необходимо было перемешать 540 тысяч строк.

#### Порядок `details`, `games`, `tags`

1. после соединения `details` с предварительно отфильтрованной таблицей `games` по признаку `mac` получится 145 тысяч строк,
1. результат предыдущего соединения соединяется с `tags`.

В результате получится 145 тысяч строк, но на промежуточном этапе необходимо было перемешать 145 тысяч строк.

```
540K / 145K = 3.72
```

**Уменьшение интернет трафика в 3.7 раза!**

#### Порядок `games`, `tags`, `details`

1. `games` и `tags` необходимо соединять декартовым произведением, т.к. у них нет общих колонок, получится 5.3 миллиона строк,
1. результат предыдущего соединения соединяется с `games`.

В результате получится 145 тысяч строк, но на промежуточном этапе необходимо было перемешать 5 миллионов строк.

**Увеличение трафика в 10 раз по сравнению с первым вариантом!**

![](../imgs/spark-join-order.drawio.svg)

#### Ответ

Наиболее оптимальным порядком соединения является: (`details` + `games`) + `tags`.

Таким образом, нужно стремиться к тому, чтобы на более ранних этапах отсекать как можно большее число строк

> **Отсечение наибольшего числа строк на более ранних этапах - залог быстродействия запроса**

### Задание

1. Выполнить предыдущие три запроса с разным порядком таблиц в соеднении;
1. Убедиться, что теоретические цифры соответствуют практическим цифрам.

### Вывод

По умолчанию Apache Spark не применяет оптимизаций, связанных с порядком соединения таблиц, а значит пользователь должен сам решать какой порядок покажет наилучшую производительность.

## Автоматический выбор наилучшего порядка соединения таблиц

В Apache Spark 2.2 появился оптимизатор на базе стоимости (Cost Based Optimizer, CBO), который при помощи статистики может самостоятельно определить наилучший порядок соединения таблиц. По умолчанию он выключен:

In [ ]:
print(f'Статус Cost Based Optimizer: {"включен" if spark.conf.get("spark.sql.cbo.enabled") == "true" else "выключен"}')

Основные задачи Cost Based Optimizer (CBO):

- сбор, вывод и распространение статистики по таблицам, колонкам и промежуточным данными,
- вычисление стоимости каждого оператора в терминах результирующего количества строк и размера итогового датасета,
- выбора наилучшего плана исполнения на базе стоимости вычислений.

### Активация CBO

Для включения оптимизатора на базе стоимости необходимо активировать опцию: `spark.sql.cbo.enabled`

In [ ]:
spark.conf.set("spark.sql.cbo.enabled", True)

In [ ]:
print(f'Статус Cost Based Optimizer: {"включен" if spark.conf.get("spark.sql.cbo.enabled") == "true" else "выключен"}')

**Замечание:** Cost Based Optimizer можно включать и выключать во время работы приложения сколько угодно раз.

### Активация автоматического поиска оптимального порядка таблиц

Для активации автоматического определения наилучшего порядка соединения таблиц в дополнении к опции `spark.sql.cbo.enabled` необходимо активировать еще опцию `spark.sql.cbo.joinReorder.enabled`:

In [ ]:
spark.conf.set("spark.sql.cbo.joinReorder.enabled", True)

Поиск оптимального порядка таблиц в соединении выполняется при помощи динамического программирования, алгоритмы которого обычно имеют квадратичную сложность, поэтому авторы CBO ограничили количество таблиц в запросе числом 12: если таблиц больше, то CBO не будет определять наилучшй порядок. Чтобы изменить это ограничение, необходимо указать желаемое число таблиц в опции `spark.sql.cbo.joinReorder.dp.threshold`:

In [ ]:
spark.conf.set("spark.sql.cbo.joinReorder.dp.threshold", 12)

### Активация статистики

CBO активно использует статистику, поэтому необходимо наилучшим образом ее подготовить. Для этого необходимо активировать еще две опции:

- `spark.sql.statistics.histogram.enabled` для сбора статистики в виде гистограмы по каждой колонке,
- `spark.sql.statistics.size.autoUpdate.enabled` для автоматического обновления статистики, когда появляются новые данные.

In [ ]:
spark.conf.set("spark.sql.statistics.histogram.enabled", True)
spark.conf.set("spark.sql.statistics.size.autoUpdate.enabled", True)

### Особенность работы CBO

Особенностью работы CBO является тот факт, что поиск оптимального порядка таблиц может выполняться только при использовании Spark SQL: DataFrame API не поддерживается

> **CBO не работает с DataFrame API!**

### Создание таблиц

Для сравнения с предыдущими экспериментами необходимо создать три таблицы: `games`, `details`, `tags`:

In [ ]:
spark.sql("DROP TABLE IF EXISTS games")
spark.sql("DROP TABLE IF EXISTS details")
spark.sql("DROP TABLE IF EXISTS tags")

In [ ]:
games.write.mode("overwrite").saveAsTable("games")
details.write.mode("overwrite").saveAsTable("details")
tags.write.mode("overwrite").saveAsTable("tags")

### Сбор статистики

Сбор статистики выполняется как на уровне таблицы, так и на уровне колонок. Статистика сохраняется в metastore (Hive, Derby).

In [ ]:
spark.sql("ANALYZE TABLE games COMPUTE STATISTICS FOR ALL COLUMNS")

Интересное наблюдение: статистика собирается немедленно.

> **Статистика собирается сразу**

### Получение статистики

При помощи `DESC EXTENDED games <column_name>` можно получить статистику по одной колонке. В результате будет показана общая статистика: минимальные, максимальные значения столбца, количество уникальных значений и т.д.

В дополнение также будет отображена гистограмма: все значения столбца разложены по определенным корзинам (bin), а для каждой корзины вычислены минимальное и максимальные значения, а также общее число элементов в корзине.

In [ ]:
spark.sql("DESC EXTENDED games app_id") \
  .repartition(1) \
  .withColumn("id", F.monotonically_increasing_id()) \
  .where(col("id").between(0, 21) | col("id").between(250, 270)) \
  .show(40, False)

Запрос ниже вычисляет минимальное и максимальное значения столбца `app_id`, которые в точности совпадают со значениями в статистике:

In [ ]:
games \
  .select(
    F.min(col("app_id")),
    F.max(col("app_id"))
  ) \
  .show()

### Сбор статистики для остальных таблиц

In [ ]:
spark.sql("ANALYZE TABLE details COMPUTE STATISTICS FOR ALL COLUMNS")
spark.sql("ANALYZE TABLE tags COMPUTE STATISTICS FOR ALL COLUMNS")

### Демонстрация работы CBO

Выполним запрос из начала ноутбука:

In [ ]:
res = spark.sql("""
  SELECT *
    FROM details
    JOIN tags USING (tag)
    JOIN games USING (app_id)
""")
res.explain()

Поменяем порядок соединения таблиц:

In [ ]:
res = spark.sql("""
  SELECT *
    FROM details
    JOIN games USING (app_id)
    JOIN tags USING (tag)
""")
res.explain()

Порядок соединения таблиц в физическом плане совпадает с предыдущим запросом. Таким образом, CBO считает наиболее оптимальным порядком соединения таблиц:

1. соединить `details` и `tags`,
1. соединить результат с `games`.

### Теоретическое обоснование пользы статистики

Статистика собирается как для всей таблицы, так и для каждой колонки. Собранная статистика по колонке раскладывается по корзинам (bin). У каждой корзны есть своя статистика, что позволяет отбрасывать корзины и строки, которые в них попали, при анализе запросов с фильтрами (`<`, `>`, `=`, `!=`, `BETWEEN`, `IN` и т.д.)

Особенность корзин заключается в том, что они хранят примерно одинаковое число значений, т.е. пространство значений столбца разбивается на неравные промежутки.

На картинке ниже приведен пример:

- тип элементов: `long`,
- количество значений в колонке: 17
- количество корзин гистограммы: 4
- среднее (медиана) число элементов в корзине: 4

![](../imgs/spark-statistics-bins.drawio.svg)

#### Сравнение колонки с константой

Если имеется статистика, то для оценки числа строк в результате при сравнении колонки с константой, например, `A < 42`, возможны три варианта:

1. минимальное значение колонки больше константы => ни одной строки не будет получено в результате,
2. константа лежит в интервале от минимального до максимального значения колонки => количество строк в результате как процент от общего числа строк,
3. константа больше максимального значения колонки => все строки попадают в результат.

На картинке ниже можно увидеть результат сравнения колонки с константой:

![](../imgs/spark-statistics-in-action-const.drawio.svg)

#### Сравнение двух колонок

Если сравниваются две колонки (`A < B`), то возможны 4 варианта:

1. максимальное значение `A` меньше минимального значения `B` => все строки попадают в результирующее множество,
2. максимальное значение `B` меньше минимального значения `A` => пустое множество,
3. перекрытие множеств:
    - минимальное значение `B` в интервале между минимальным и максимальным значениями `A` => в результате процент от общего числа строк,
    - максимальное значение `B` в интервале между минимальным и максимальным значениями `A` (частный случай - вложение `B` в `A`) => в результате процент от общего числа строк.


![](../imgs/spark-statistics-in-action-column.drawio.svg)

#### Общее число строк при соединении (JOIN)

Для оценки соединения (join) двух таблиц:

```sql
SELECT *
  FROM T
  JOIN U ON (T.k1 = U.k1)
```

можно оценить общее число строк в результате следующей формулой:

$$
|T \Join U| = \frac{|T| * |U|} {max(|T.k1|, |U.k1|)}
$$

где:

- `|T|` - число строк в таблице `T`,
- `|T.k1|` - число уникальных значений столбца `k1` в таблице `T`.

### Вывод

Поиск налучшего порядка соединения таблиц может быть нетривиальной задачей, особенно, когда запрос соедржит большое число таблиц. Оптимизатор на базе стоимости позволяет в автоматическом режиме определять наилучший порядок соединения таблиц, но работает он только в Spark SQL. Несмотря на это ограчение, можно использовать CBO для сравнения двух вариантов одного запроса для выработки интуиции при оптимизации запросов.

### Задание

Добавить различные фильтры `WHERE` в запросы с JOIN и проанализировать планы запросов

## Оптимизация на базе hint инструкций

### Типы соединений (JOIN)

Существуют несколько алгоритмов соединения таблиц. Далее перечислены все алгоритмы от быстрого к медленному:

| Имя | Описание | Особенности |
|-----|----------|-------------|
| BROADCAST | Скопировать наименьший по размеру датафрейм на каждый воркер и выполнить соединение с теми партициями, которые уже находятся на воркерах | Датафрейм не может превышать 10МБ (можно настроить), а также нужно следить за памятью. При увеличении числа воркеров, производительность запроса снижается (Почему?) |
| HASH | Строки наименьшего по размеру датафрейма разложить по воркерам в HashMap, строки второго датафрейма отправить на воркеры и положить их в HashMap, если такой ключ существует. Из HashMap сформировать результирующий датафрейм | HashMap может занять очень много места в памяти, что может привести к OutOfMemoryError |
| MERGE | Оба датафрейма сортируются и отправляются (shuffle) на воркеры. На каждом воркере хранится часть каждого датафрейма. После того, как все строки обоих датафреймов отправлены, на каждом воркере запускается merge алгоритм аналогичный [Алгоритму Сортировки Слиянием](https://ru.wikipedia.org/wiki/%D0%A1%D0%BE%D1%80%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B0_%D1%81%D0%BB%D0%B8%D1%8F%D0%BD%D0%B8%D0%B5%D0%BC) | Изначально данные сохраняются в раздел "исполнение" (execution) памяти воркера, но если места перестает хватать, то Apache Spark начинает выполнять спилы. Так оба датафрейма могут попасть на диск, что сильно снизит производительность алгоритма. Работает на данных любого размера. Принят в качестве алгоритма соединения (join) по умолчанию |
| NESTED LOOP | Декартово произведение | Самый медленный, на практике не используется |

> **MERGE JOIN - алгоритм соединения по умолчанию**

### BROADCAST JOIN

На картинке ниже представлен `BROADCAST JOIN`, который выполняется за две стадии:

1. **Broadcast Exchange** - отправить все партиции небольшого датафрейма на все воркеры,
2. **Broadcast Merge Join** - выполнить локальную операцию соединения (join) на каждом воркере одновременно.

![](../imgs/spark-broadcast-join.drawio.svg)

### HASH JOIN

На картинке ниже представлены основные стадии HASH JOIN, которые включают в себя:

1. перемешивание (Exchange, Shuffle) меньшего по размеру датафрейма по ключу соединения (JOIN),
2. создание HashMap на базе полученных данных,
3. перемешивание (Exchange, Shuffle) второго датафрейма с немедленным отсечением строк, для которых нет значения в созданном на предыдущем шаге к HashMap по ключу соединения (join),
4. создание результирующего датафрейма на базе HashMap по записям, для которых нашлись пары в обоих датафреймах по ключу хеширования/соединения.

![](../imgs/spark-hash-join.drawio.svg)

### SORT MERGE JOIN

На картинке ниже представлены основные стадии Sort Merge Join:

- перемешивание обоих датафреймов по ключу соединения,
- сортировка на каждом воркере одновременно,
- слияние отсортированных датасетов (time: `O(N)`, space: `O(1)`),
- создание результрующего датафрейма по ключам, которые нашлись в обоих отсортированных датафреймах.

![](../imgs/spark-sort-merge-join.drawio.svg)

### Настройка типа соединения (JOIN)

Как было указано выше Apache Spark использует Sort Merge JOIN в качестве алгоритма по умолчани, т.к. он позволяет гаратировано обработать данные любого размера.

Но на датафреймах небольшого размера предпочтительно использовать HASH JOIN, если есть гарантии, что этот датафрейм не будет расти.

Для того, чтобы указать тип соединения, можно воспользоваться [hint](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.hint.html)-инструкциями (хинтами)

В качестве примера рассмотрим следующий запрос:

In [ ]:
details.join(tags, "tag").explain()

План запроса показывает, что Apache Spark будет выполнять его при помощи `SortMergeJoin`. Учитывая, что датафрейм `tags` является небольшим и вряд ли в будущем будет сильно расти, можно выбрать HASH JOIN:

In [ ]:
details.join(tags.hint("SHUFFLE_HASH"), "tag").explain()

В плане выше можно заметить, что Apache Spark будет выполнять этот запрос при помощи `ShuffledHashJoin`

Можно пойти еще дальше запросить BROADCAST join:

In [ ]:
tags.hint("BROADCAST").join(details, "tag").explain()

### Настройка соединений в Spark SQL

Хинты можно применять и в Spark SQL:

In [ ]:
spark.sql("SELECT * FROM details JOIN tags USING (tag)").explain()

Apache Spark использует `SortMergeJoin` по умолчанию. Чтобы выбрать другой алгоритм необходимо указать его в хинте:

In [ ]:
spark.sql("SELECT /*+ SHUFFLE_HASH(t) */ * FROM details d JOIN tags t USING (tag)").explain()

In [ ]:
spark.sql("SELECT /*+ BROADCAST(t) */ * FROM details d JOIN tags t USING (tag)").explain()

In [ ]:
spark.sql("SELECT /*+ SHUFFLE_REPLICATE_NL(t) */ * FROM details d JOIN tags t USING (tag)").explain()

### Количество промежуточных партиций

Результатом соединения таблиц будет новый датафрейм, в котором количество партиций определяется значением опции `spark.sql.shuffle.partitions`

In [ ]:
print(f'После JOIN любой датафрейм будет иметь {spark.conf.get("spark.sql.shuffle.partitions")} партиций')

Нужно учитывать тот факт, что некоторые партиции могут быть пустыми.

Значение по умолчанию принято 200, но на практике почти всегда это значение является неверным и можно получить результат быстрее, если это значение изменить.

### Как определить нужное число партиций?

Оптимальное значение `spark.sql.shuffle.partitions` нужно подбирать под каждый запрос, т.е. нужно добавлять `spark.conf.set("spark.sql.shuffle.partitions", X)` перед запросом, где `X` вычисляется по следующей формуле:

$$
X = \frac {Stage Input Data} {200МБ}
$$

, где

- $StageInputData$ - объем данных на входе стадии,
- `200МБ` - оптимальный объем партиции в Apache Spark.

**Дано**:

- 100 воркеров,
- 32ГБ оперативной памяти на воркер,
- 20 CPU на воркере,
- размер данных на входе (Stage Input Data) = 210ГБ.

**Найти**: оптимальное значение для `spark.sql.shuffle.partitions`

**Решение**

Примем за оптимальный размер партиции 200МБ.

Тогда:

```
210ГБ / 200МБ = 1050 партиций всего
```

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", 1050)

Но у нас в кластере количество CPU равно 2000 ($100\spaceворкеров \times 20\space CPU$), поэтому нужно установить значение 2000, чтобы равномерно загрузить кластер:

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", 2000)

## Оптимизация перекошенных (ассиметричных, skew) партиций

Перекос в данных по партициям сильно вредит производительности запроса, потому что:

- вся работа разбивается на стадии (stage), которые выполняются друг за другом,
- стадия (stage) разбивается на задания (task),
- одно задание соответствует одной партиции датафрейма,
- за один раз один воркер может обработать одно задание,
- каждое задание (task) стадии должно быть обработано, чтобы стадия завершилась,
- следующая стадия должна дождаться выполнения всех заданий предыдущей стадии.

Таким образом, если существует очень большая партиция, то она будет препятствовать скорейшему завершению стадии.

На картинке ниже приведен пример датафрейма с перекосом в распределении строк в пользу партиции `P1`:

![](../imgs/spark-skew-partitions.drawio.svg)

При наличии трех воркеров обработка датафрейма на картике выше будет выглядеть следующим образом:

![](../imgs/spark-skew-partitions-processing.drawio.svg)

Из картики видно, что большую часть времени два воркера простаивало, т.к. необходимо было дождаться завершения обрабтки партиции `P1`

Интуитивно можно догадаться, что необходимо разбить большую партицию на части и обрабатывать их одновременно:

![](../imgs/spark-skew-partitions-processing-fix.drawio.svg)

Существует два способа борьбы с таким классом проблем при соединениях (join):

- использование AQE,
- ручная подготовка ключа (Salting).

### Использование AQE

#### Описание

В Apache Spark 3.0 появился адаптивный оптимизатор (Adaptive Query Execution, AQE), который умеет автоматически обрабатывать проблемы с перекосом. Он активирован по умолчанию, но чтобы убедиться в этом необходимо выполнить следующую инструкцию:

In [ ]:
active = "Включена" if spark.conf.get("spark.sql.adaptive.skewJoin.enabled") == "true" else "Выключена"
print(f'Автоматическая обработка перекошенных партиций: {active}')

Если автоматическая обработка перекошенных партиций выключена, то включить ее можно таким образом:

In [ ]:
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", True)

In [ ]:
active = "Включена" if spark.conf.get("spark.sql.adaptive.skewJoin.enabled") == "true" else "Выключена"
print(f'Автоматическая обработка перекошенных партиций: {active}')

Но просто активировать опцию `spark.sql.adaptive.skewJoin.enabled` недостаточно, необходимо активировать сам AQE:

In [ ]:
spark.conf.set("spark.sql.adaptive.enabled", True)

По умолчанию AQE будет считать перекошенными не все партиции, а только те, которые выбиваются из ограничений:

- партиция больше медианного (среднего) размера партиций в 5 раз,
- партиция больше 250МБ в размере.

Если оба ограничения должны быть превышены, то партиция будет считаться перекошенной.

Оба ограничения можно настраивать:

- `spark.sql.adaptive.skewJoin.skewedPartitionFactor` контролирует во сколько раз партиция должна быть больше средней, чтобы считаться перекошенной (skew),
- `spark.sql.adaptive.skewJoin.skewedPartitionThresholdInBytes` контролирует минимальный размер партиции для перекоса.

На небольших данных может быть эффективнее обработать как есть, вместо того, чтобы активировать автоматическую обработку перекоса.

AQE может добавить дополнительную Exchange операцию, но при этом Catalyst может найти более оптимальный план без дополнительной операции перемешивания, поэтому оптимизация skew join может не сработать. В версии Apache Spark 3.3 добавили дополнительную настройку `spark.sql.adaptive.forceOptimizeSkewedJoin`, которая вынудит Spark использовать план с дополнительной операцией Exchange.

**Внимание**: Оптимизация перекошенных партиций в Apache Spark находится в зачаточном состоянии, поэтому может не всегда работать. Например, если в запросе участвуют кэши, то оптимизация перекошенных партиций при помощи AQE не сработает.

### Ручная подготовка ключа

#### Описание

Традиционным способом борьбы с перекошенными партициями является ручная подготовка ключа (salting). Идея следующая:

- в соединении (join) участвуют две таблицы: "правая" и "левая";
- в "левой" таблице для колонки соединения (в ней перекосом по данным) создается новая колонка с суффиксом `_1`, `_2`, `_3` и т.д. из заранее подготовленного списка суффиксов;
- в "правой" таблице для колонки из соединения (в ней перекос по данным), создается новая колонка типа array, где перечислены все возможные значения колонки с суффиксом из левой таблицы;
- правая таблица "взрывается" (`explode`) копиями по созданной колонке типа array;
- соединение выполняется по новой колонке с двух сторон.

![](../imgs/spark-salted-join.drawio.svg)

На картике выше цветом отображаются:

- одинаковые значения `app_id`,
- воркер на который попадет строка после перемешивания по `app_id_salted`.

В целях облегчения восприятия картинки, строки со значениями 5, 9, 11, не были отображены на воркерах после перемешивания, т.к. они не попадут в финальный результат. Но для единообразия цветом показано на какие воркеры они бы попали.

#### Демонстрация

Соединение (join) таблицы `details` и `games` будет выполняться по колонке `app_id`:

- `details` - левая таблица,
- `games` - правая таблица.

В "левую" таблицу `details` необходимо добавить новую колонку: `app_id_salted`, которая будет конкатенацией колонки `app_id` со случайным префиксом `_1`, `_2` или `_0`:

In [ ]:
details_salted = details \
    .withColumn(
        "app_id_salted",
        F.concat(
            col("app_id"),
            F.lit("_"),
            F.floor(F.rand() * 3))
    )
details_salted.printSchema()

В правую таблицу необходимо добавить колонку типа "array", а потом "взорвать" датафрейм по ней, чтобы получить дубликаты строк с колонкой `app_id_salted`, которая имеет все возможные значения суффикса:

In [ ]:
app_ids_salted = [
    F.concat(col("app_id"), F.lit("_"), F.lit(x))
    for x in range(3)
]

games_salted = games \
    .withColumn(
        "app_id_salted",
        F.explode(F.array(app_ids_salted))
    )

games_salted.printSchema()

Теперь вместо `app_id` ключом соединения (join) будет `app_id_salted`. После соединения лишние колонки нужно удалить:

In [ ]:
games_details = details_salted.join(games_salted, "app_id_salted") \
    .drop(games_salted.app_id) \
    .drop(games_salted.app_id_salted)
games_details.printSchema()

In [ ]:
games_details.explain()

Обратите внимание, что [`explode`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html) является локальной операцией, поэтому необходимо убедиться, что "взрываться" будет меньшая по размеру таблица.

### Вывод

Несмотря на наличие AQE и его возможности по оптимизации перекошенных партиций, его возможности все ещё достаточно ограничены. Хотя ручная подготовка ключа и даст больший контроль над процессом, но оптимизатор Apache Spark постоянно улучшается и в будущем его возможности будут превосходить результаты с явной подготовкой ключа, т.к. это правило оптимизации будет работать вместе со всеми остальными правилами, что даст лучший эффект. Поэтому рекомендуется, периодически возвращаться к запросам с ручной подготовкой ключа после каждого нового релиза, и проверять можно ли отказаться от ручного способа подготовки ключа.

### Задание

Выполнить соединение:

```sql
SELECT *
  FROM details d
  JOIN tags t ON (d.tag = t.tag)
```

при помощи ручной подготовки ключа с использованием:

- DataFrame API,
- Spark SQL.